# AutoGen Tool Use Example 

## Import the Needed Packages 

In [1]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

In [2]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

## Creating the Client 

In this sample, we will use [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) for access to the LLM. 

The `model` is defined as `gpt-4o-mini`. Try changing the model to another model available on the GitHub Models marketplace to see the different results. 

As a quick test, we will just run a simple prompt - `What is the capital of France`. 

In [3]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

c:\Users\danny\miniconda3\envs\azure_tuto\Lib\site-packages\autogen_ext\models\azure\_azure_ai_client.py:305: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(config["model_info"])


finish_reason='stop' content='The capital of France is Paris.' usage=RequestUsage(prompt_tokens=14, completion_tokens=8) cached=False logprobs=None thought=None


## Defining the Functions 

In this example, we will give the agent access to a tool that is a function with a list of available vacation destinations and their availability. 

You can think that this would be a scenario where a travel agent might have an access to a travel database for example. 

As you go through this sample, feel free to try to define new functions and tools that the agent can call. 

In [4]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Seoul": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
city, status = vacation_destinations("Cape Town")
print(f"How about visiting {city}? It's currently {status} there!")

How about visiting Cape Town? It's currently City not found there!


## Defining the Function Tool 
To have the agent use the `vacation_destinations` as a `FunctionTool`, we need to define it as one. 

We will also provide a description of the to tool which is helpful for the agent to identify what that tool is used for in relation to the task the user has requested. 

In [5]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## Defining the agent 

Now we can create the agent in the below code. We define the `system_message` to give the agent instructions on how to help users find vacation destinations. 

We also set the `reflect_on_tool_use` parameter to true. This allows use the LLM to take the response of the tool call and send the response using natural language. 

You can set the the parameter to false to see the difference. 

In [6]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## Running the Agent 

Now we can run the agent with the initial user message asking to take a trip to Tokyo. 

You can change this city desintation to see how the agent responds to the availablity of the city. 

In [7]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Seoul", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()

[ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=74, completion_tokens=19), metadata={}, content=[FunctionCall(id='call_hq2h9WbNKK8fwpiJius3Bb8J', arguments='{"city":"Seoul"}', name='vacation_destinations')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant', models_usage=None, metadata={}, content=[FunctionExecutionResult(content="('Seoul', 'Available')", name='vacation_destinations', call_id='call_hq2h9WbNKK8fwpiJius3Bb8J', is_error=False)], type='ToolCallExecutionEvent')]
source='assistant' models_usage=RequestUsage(prompt_tokens=67, completion_tokens=325) metadata={} content="Seoul is a fantastic destination with a mix of modern skyscrapers, traditional palaces, vibrant culture, and delicious cuisine. Here are some highlights and tips for your trip:\n\n### Top Attractions:\n1. **Gyeongbokgung Palace** - This majestic palace is a must-visit. Don't miss the changing of the guard ceremony!\n2. **Bukchon Hanok Village** - Wander th